In [ ]:
import os
import numpy as np
import typing
from osgeo import gdal
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive/')

def gtiff_to_array(file_path):
    """Takes a file path and returns a tif file as a 3-dimensional numpy array, width x height x bands."""
    data = gdal.Open(file_path)
    bands = [data.GetRasterBand(i+1).ReadAsArray() for i in range(data.RasterCount)]
    return np.stack(bands, axis=2), data.GetGeoTransform(), data.GetProjection()

def export_geotiff(labels: np.ndarray, geoTrans, geoProj, filename: str):
    """Maps labels onto the colors black & white and exports the resulting image
    as a file with the given file.

    :param labels: 2D array of labels per pixel
    :param filename: File name of the new file
    """
    x, y = labels.shape
    # labels[:, :] = np.where(labels[:, :] == 2, 65535, 0)
    labels[:, :] *= 65535
    # import matplotlib.pylab as plt
    # plt.hist(labels.reshape(x*y)); plt.show()
    driver = gdal.GetDriverByName("GTiff")
    outData = driver.Create(filename, x, y, 3, gdal.GDT_UInt16)

    # Write metadata
    outData.SetGeoTransform(geoTrans)
    outData.SetProjection(geoProj)

    # Write raster data sets
    for i in range(3):
      outData.GetRasterBand(i+1).WriteArray(labels)
    outData.FlushCache()
    outData = None

dataWithGeo = 'prediction_vietnam_180.tif'
dataWithoutGeo = 'WST_8000_26_nongeo.tif'
BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/Agricultural_Field_Boundary/add_location"

data0, geoTrans0, geoProj0 = gtiff_to_array(BASE_PATH + '/' + dataWithGeo)
print(geoTrans0, geoProj0)

data1, geoTrans1, geoProj1 = gtiff_to_array(BASE_PATH + '/' + dataWithoutGeo)
print(geoTrans1, geoProj1)
export_geotiff(data1[:,:,0], geoTrans0, geoProj0, BASE_PATH + '/' + dataWithoutGeo)


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
(612696.875, 3.125, 0.0, 2294321.875, 0.0, -3.125) PROJCS["WGS 84 / UTM zone 48N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",105],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32648"]]
(0.0, 1.0, 0.0, 0.0, 0.0, 1.0) 


# New Section